# SAÉ: Labyrinthes

## Implémentation

In [29]:
from random import *

### Classe `Maze`

In [78]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1, c2):
        """
        Ajout d'un mur entre les cellules c1 et c2
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : \
            les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c1 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1, c2):
        """
        Suppression du mur entre les cellules c1 et c2
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : \
            les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        
        # Suppression du mur
        if c2 not in self.neighbors[c1]:   # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2)     # on l'ajoute
        if c1 not in self.neighbors[c2]:   # Si c1 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1)     # on l'ajoute
            
    def empty(self):
        """
        Suppression de tous les murs du labyrinthe
        """
        # On ajoute pour chaque cellule ses cellles voisines (haut, droite, bas, gauche) 
        # dans son voisinage si cela est possible
        
        for i in range(self.height):
            for j in range(self.width):
                # Ajout de la cellule du haut
                if i > 0:                    # Si le haut de la cellule n'est pas la limite du labyrinthe
                    if (i-1, j) not in self.neighbors[(i, j)]:  # Si la cellule du haut n'est pas déja voisine
                        self.neighbors[(i, j)].add((i-1, j))    # on l'ajoute
                        
                # Ajout de la cellule de droite
                if j < self.width - 1:       # Si la droite de la cellule n'est pas la limite du labyrinthe
                    if (i, j+1) not in self.neighbors[(i, j)]:  # Si la cellule de droite n'est pas déja voisine
                        self.neighbors[(i, j)].add((i, j+1))    # on l'ajoute
                        
                # Ajout de la cellule du bas
                if i < self.height - 1:      # Si le bas de la cellule n'est pas la limite du labyrinthe
                    if (i+1, j) not in self.neighbors[(i, j)]:  # Si la cellule du bas n'est pas déja voisine
                        self.neighbors[(i, j)].add((i+1, j))    # on l'ajoute
                        
                # Ajout de la cellule de gauche
                if j > 0:                    # Si la gauche de la cellule n'est pas la limite du labyrinthe
                    if (i, j-1) not in self.neighbors[(i, j)]:  # Si la cellule du haut n'est pas déja voisine
                        self.neighbors[(i, j)].add((i, j-1))    # on l'ajoute
                        
    def fill(self):
        """
        Ajout de tous les murs du labyrinthe
        """
        # On supprime pour chaque cellule ses cellles voisines (haut, droite, bas, gauche) 
        # de son voisinage si cela est possible
        
        for i in range(self.height):
            for j in range(self.width):
                # Suppression de la cellule du haut
                if i > 0:                    # Si le haut de la cellule n'est pas la limite du labyrinthe
                    if (i-1, j) in self.neighbors[(i, j)]:  # Si la cellule du haut est voisine
                        self.neighbors[(i, j)].remove((i-1, j))    # on la retire
                        
                # Suppression de la cellule de droite
                if j < self.width - 1:       # Si la droite de la cellule n'est pas la limite du labyrinthe
                    if (i, j+1) in self.neighbors[(i, j)]:  # Si la cellule de droite est voisine
                        self.neighbors[(i, j)].remove((i, j+1))    # on la retire
                        
                # Suppression de la cellule du bas
                if i < self.height - 1:      # Si le bas de la cellule n'est pas la limite du labyrinthe
                    if (i+1, j) in self.neighbors[(i, j)]:  # Si la cellule du bas est voisine
                        self.neighbors[(i, j)].remove((i+1, j))    # on la retire
                        
                # Suppression de la cellule de gauche
                if j > 0:                    # Si la gauche de la cellule n'est pas la limite du labyrinthe
                    if (i, j-1) in self.neighbors[(i, j)]:  # Si la cellule du haut est voisine
                        self.neighbors[(i, j)].remove((i, j-1))    # on la retire
                        
    def get_walls(self):
        """
        Retourne tous les murs présents dans le labyrinthe.
        
        :return: Liste contenant les murs caractérisés par une liste de tuples représentant les cellules.
        """
        # Liste qui stockera les murs
        walls = []
        
        # Le but ici est de vérifier si un mur est présent uniquement sur à l'EST et au SUD de chaque cellule.
        
        for i in range(self.height):
            for j in range(self.width):
                # On vérifie si un mur est présent à droite et donc si la cellule de droite n'est pas voisine
                if j < self.width - 1 and (i, j+1) not in self.neighbors[(i, j)]:
                    walls.append([(i, j), (i, j+1)])  # On ajoute le mur à la liste des murs
                    
                # On vérifie si un mur est présent en bas et donc si la cellule du bas n'est pas voisine
                if i < self.height - 1 and (i+1, j) not in self.neighbors[(i, j)]:
                    walls.append([(i, j), (i+1, j)])  # On ajoute le mur à la liste des murs
                    
        return walls
    
    def get_contiguous_cells(self, c):
        """
        Retourne les cellules contigües à c.
        
        :return: Liste des cellules contigües à c.
        """
        # Liste des cellules contigües à c.
        contiguous_cells = []
        
        if c[0] > 0:                                 # Si la cellule n'est pas sur la première ligne
            contiguous_cells.append((c[0]-1, c[1]))  # on ajoute la cellule du haut
            
        if c[0] < self.height - 1:                   # Si la cellule n'est pas sur la denière ligne
            contiguous_cells.append((c[0]+1, c[1]))  # on ajoute la cellule du bas
            
        if c[1] > 0:                                 # Si la cellule n'est pas sur la première colonne
            contiguous_cells.append((c[0], c[1]-1))  # on ajoute la cellule de gauche
        
        if c[1] < self.width - 1:                    # Si la cellule n'est pas sur la dernière colonne
            contiguous_cells.append((c[0], c[1]+1))  # on ajoute la cellule du haut
        
        return contiguous_cells
    
    def get_reachable_cells(self, c):
        """
        Retourne toutes les cellules contigües et voisines à c.
        """
        reachable_cells = []
        
        for cell in self.get_contiguous_cells(c): # Parmi toutes les cellules contigües à c
            if cell in self.neighbors[c]:         # Si elles sont voisines à c
                reachable_cells.append(cell)      # on l'ajoute dans la liste des cellules atteignables
                
        return reachable_cells
    
    
    @classmethod
    def gen_btree(cls, h, w):
        """
        Génère un labyrinthe de h lignes et w colonnes
        en utilisant l'algorithme de construction par arbre binaire.
        """
        # On génère le labyrinthe plein grâce au constructeur
        maze = cls(h, w)
        
        # Ensuite, pour chaque cellule, on suppirmera:
        #  - aléatoirement le mur EST ou le mur SUD
        #  - s'il n'y a qu'un seul de ces deux murs, on le supprimera
        #  - auucn mur s'il n'y a aucun de ses murs
        
        for i in range(h):
            for j in range(w):
                # Il ne peut y avoir des murs à l'est et au sud uniquement aux cellules
                # qui ne sont ni sur la dernière ligne ni sur la dernière colonne.
                if i < h - 1 and j < w - 1:
                    # on supprime aléatoirement le mur au sud ou à l'est
                    if randint(0, 1) == 1:            # suppression du mur est
                        maze.remove_wall((i, j), (i, j+1))
                    else:                             # suppression du mur sud
                        maze.remove_wall((i, j), (i+1, j))
                        
                # Le cas où il n'y peut avoir qu'un mur au sud est sur la dernière colonne
                # excepté la dernière cellule
                elif j == w - 1 and i < h - 1 :
                    maze.remove_wall((i, j), (i+1, j)) # suppression du mur sud
                    
                # Le cas où il n'y peut avoir qu'un mur à l'est est sur la dernière ligne
                # excepté la dernière cellule
                elif i == h - 1 and j < w - 1 :
                    maze.remove_wall((i, j), (i, j+1)) # suppression du mur est
                    
        return maze
    
    
    @classmethod
    def gen_sidewinder(cls, h, w):
        """
        Génère un labyrinthe de h lignes et w colonnes
        en utilisant l'algorithme de construction sidewinder.
        """
        # On génère le labyrinthe plein grâce au constructeur
        maze = cls(h, w)
        
        for i in range(h - 1):           # Pour i allant de 0 à hauteur-2
            seq = []                     # Initialiser une variable séquence comme liste vide
            for j in range(w - 1):       # Pour j allant de 0 à largeur-2
                seq.append((i, j))       # Ajouter la cellule (i, j) à la séquence
                
                # Tirage pile ou face
                if randint(0, 1) == 1:                   # Si c'est pile
                    maze.remove_wall((i, j), (i, j+1))   # suppression du mur EST
                else:                                    # Si c'est face
                    random_cell = choice(seq)            # On choisit au hasard une cellule de la séquence
                    maze.remove_wall(random_cell, (random_cell[0] + 1, random_cell[1])) # suppression du mur SUD
                    seq = []                             # Réinitialiser la séquence à une liste vide
                
            seq.append((i, w - 1))                      # Ajout de la dernière cellule à la séquence
            
            random_cell = choice(seq)            # On choisit au hasard une cellule de la séquence

            maze.remove_wall(random_cell, (random_cell[0] + 1, random_cell[1])) # suppression du mur SUD
            
        for j in range(w - 1): # suppression du mur EST sur les cellules de la dernière ligne
            maze.remove_wall((h-1, j), (h-1, j+1))
            
        return maze
    
    
    @classmethod
    def gen_fusion(cls, h, w):
        """
        Génère un labyrinthe de h lignes et w colonnes
        en utilisant l'algorithme par fusions de chemins.
        """
        # On génère le labyrinthe plein grâce au constructeur
        maze = cls(h, w)
        
        # On labélise les cellules de 0 à n-1. Pour chaque cellule, on mettra son label corresponant
        # dans une liste.
        labels = []
        for i in range(h * w):
            labels.append(i)
        
        # On extrait la liste de tous les murs et on les mélange
        walls = maze.get_walls()
        shuffle(walls)
        
        # Pour chaque mur de la liste
        for wall in walls:
            # On enregistre les labels des cellules séparées par le mur
            label_cell_1 = labels[wall[0][0] * w + wall[0][1]]
            label_cell_2 = labels[wall[1][0] * w + wall[1][1]]
            
            # Si les cellules n'ont pas le même label
            if label_cell_1 != label_cell_2:
                # On casse le mur
                maze.remove_wall(wall[0], wall[1])
                
                # On affecte le label de la cellule ayant le plus petit label
                # à l'autre cellule et à toutes cellules ayant le même label que la deuxième
                main_label = -1
                secondary_label = -1
                
                if label_cell_1 < label_cell_2:
                    labels[wall[1][0] * w + wall[1][1]] = label_cell_1
                    # Label "principal" qui prend le dessus lors de la fusion
                    main_label = label_cell_1
                    # Label "secondaire" qui va être remplacé par le premier label
                    secondary_label = label_cell_2
                else:
                    labels[wall[0][0] * w + wall[0][1]] = label_cell_2
                    main_label = label_cell_2
                    secondary_label = label_cell_1
                    
                # Affectation du label "principal" aux cellules ayant le même label que le label "secondaire"
                for i in range(len(labels)):
                    if labels[i] == secondary_label:
                        labels[i] = main_label
        
        return maze
    
    
    @classmethod
    def gen_exploration(cls, h, w):
        """
        Génère un labyrinthe de h lignes et w colonnes
        en utilisant l'algorithme de génération par exploration.
        """
        # On génère le labyrinthe plein grâce au constructeur
        maze = cls(h, w)
        
        # On choisit une cellule au hasard
        random_cell = (randint(0, h-1), randint(0, w-1))
        # On la marque comme étant visitée
        visited_cells = [random_cell]
        # et on la met dans une pile
        pile = [random_cell]
        
        # Tant que la pile n'est pas vide
        while pile:
            
            # On prend la celulle en haut de la pile et on la retire
            up_cell = pile[len(pile) - 1]
            pile.pop()
            
            # On stocke les voisins de la cellule qui n'ont pas été visitées
            contiguous_cells = maze.get_contiguous_cells(up_cell)
            not_visited_cells = []
            for cell in contiguous_cells:
                if cell not in visited_cells:
                    not_visited_cells.append(cell)

            # Si cette cellule a des voisins qui n'ont pas été visitées
            if not_visited_cells:
                # On remet cette cellule dans la pile
                pile.append(up_cell)
                
                # On choisit au hasard une des cellules contigües qui n'a pas été visitée
                random_contiguous_cell = choice(not_visited_cells)
                
                # On casse le mur entre la cellule initiale et celle tout juste choisie
                maze.remove_wall(up_cell, random_contiguous_cell)
                
                # On marque la cellule tout juste choisie comme visitée
                visited_cells.append(random_contiguous_cell)
                
                # Et on l'ajoute à la pile
                pile.append(random_contiguous_cell)
                
        return maze
    
    
    @classmethod
    def gen_wilson(cls, h, w):
        """
        Génère un labyrinthe de h lignes et w colonnes
        en utilisant l'algorithme de Wilson.
        """
        # On génère le labyrinthe plein grâce au constructeur
        maze = cls(h, w)
        
        # On crée une liste de marquqge de cellules qui sera utile pour
        # la génération du labyrinthe
        marked_cells = []
        
        # On choisit une cellule au hasard et on la marque
        marked_cells.append((randint(0, h-1), randint(0, w-1)))
        
        # Tant qu'il reste des cellules non marquées
        while len(marked_cells) < h * w:
            
            # On choisit une cellule de départ au hasard parmi les cellules non marquées
            start_cell = (randint(0, h-1), randint(0, w-1))
            while start_cell in marked_cells:
                start_cell = (randint(0, h-1), randint(0, w-1))
                
            # On effectue une "marche aléatoire" jusqu'à atteindre une cellule marquée
            random_walk = [start_cell]
            
            # Si le chemin revient sur lui-même, on annule cette "marche aléatoire"
            # et on en recommence une nouvelle
                    
            # La marche aléatoire s'arrête:
            #  - si une cellulle marquée est atteinte
            #  - si la marche forme une boucle

            loop = False

            while random_walk[len(random_walk) - 1] not in marked_cells and not loop:

                # On tire au hasard la prochaine direction de la marche
                move = choice([-1, 1])

                # On stocke les coordonnées de la prochaine cellule adjacente choisie au hasard selon la direction
                next_cell_X = 0
                next_cell_Y = 0

                if randint(0, 1) == 1:
                    next_cell_X= random_walk[len(random_walk) - 1][0] + move
                    next_cell_Y= random_walk[len(random_walk) - 1][1]
                else:
                    next_cell_X= random_walk[len(random_walk) - 1][0]
                    next_cell_Y= random_walk[len(random_walk) - 1][1] + move

                # Si la prochaine cellule est en dehors de la grille, on re-tire au hasard une nouvelle cellule
                # jusqu'à en trouver une valide
                while not (0 <= next_cell_X < w) or not (0 <= next_cell_Y < h):
                    move = choice([-1, 1])

                    next_cell_X = 0
                    next_cell_Y = 0

                    if randint(0, 1) == 1:
                        next_cell_X= random_walk[len(random_walk) - 1][0] + move
                        next_cell_Y= random_walk[len(random_walk) - 1][1]
                    else:
                        next_cell_X= random_walk[len(random_walk) - 1][0]
                        next_cell_Y= random_walk[len(random_walk) - 1][1] + move

                next_cell = (next_cell_X, next_cell_Y)

                # On regarde maintenant si la cellule a déja été visitée
                if next_cell in random_walk:
                    loop = True
                # Si ce n'est pas le cas, on peut l'ajouter à la liste de cellules visitées
                else:
                    random_walk.append(next_cell)
                    
            # Si la marche ne s'est pas arrêtée par une boucle mais car on est tombé sur une cellule marquée
            if not loop:
                
                # On marque chaque cellule du chemin
                for i in range(len(random_walk) - 1):
                    marked_cells.append(random_walk[i])
                
                # Et on casse tous les murs rencontrés jusqu'à la cellule marquée
                for i in range(len(random_walk) - 1):
                    maze.remove_wall(random_walk[i], random_walk[i + 1])
                    
        return maze
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    
    def solve_dfs(self, start, stop):
        """
        Retourne le chemin du labyrinthe reliant la cellule de départ et d'arrivée.
        Un parcours en profondeur sera utilisé ici.
        
        :params: start: Cellule de départ
        :params: stop: Cellule d'arrivée
        """
        
        # Première étape: parcourir le graphe jusqu'à trouver la cellule stop
        
        # On place la cellule de départ dans la structure d'attente, ici la pile
        pile = [start]
        # et on la marque
        marked_cells= [start]
        
        # On va maintenant mémoriser les prédécesseurs en utilisant un dictionnaire.
        # Les clés seront les cellules et les valeurs seront les cellules prédécesseurs.
        # On va mémoriser le prédécesseur de start comme étant start.
        pred = {start : start}
        
        found_stop = False
        
        # Tant qu'il reste des cellules non marquées et que l'arrivée n'a pas encore été trouvée
        while len(marked_cells) < self.height * self.width and not found_stop:
            
            # On prend la cellule en haut de la pile
            current_cell = pile[len(pile) - 1]
            # et on la retire
            pile.pop()

            # Si elle correspond à la cellule d'arrivée
            if current_cell == stop:
                # Terminé, on a trouvé un chemin vers la destination
                found_stop = True
            else:
                # Pour chaque voisine de la cellule
                cell_neighbors = self.get_reachable_cells(current_cell)
                
                for cell in cell_neighbors:
                    # Si elle n'est pas marquée, on la marque
                    if cell not in marked_cells:
                        marked_cells.append(cell)
                        # On la met dans la pile
                        pile.append(cell)
                        # Et on mémorise son prédécesseur comme étant la cellule initiale "current_cell"
                        pred[cell] = current_cell
                    
        
        # Deuxième étape: reconstruire le chemin à partir des prédécesseurs
        
        path = []
        
        # On initialise c à stop
        c = stop
        
        # Tant que c n'est pas start
        while c != start:
            # On ajoute c au chemin
            path.append(c)
            # On met le prédécesseur de c dans c
            c = pred[c]
        
        # On ajoute start dans le chemin
        path.append(start)
        
        return path
    
    
    def solve_bfs(self, start, stop):
        """
        Retourne le chemin du labyrinthe reliant la cellule de départ et d'arrivée.
        Un parcours en largeur sera utilisé ici.
        
        :params: start: Cellule de départ
        :params: stop: Cellule d'arrivée
        """
        
        # Première étape: parcourir le graphe jusqu'à trouver la cellule stop
        
        # On place la cellule de départ dans la structure d'attente, ici la file
        file = [start]
        # et on la marque
        marked_cells= [start]
        
        # On va maintenant mémoriser les prédécesseurs en utilisant un dictionnaire.
        # Les clés seront les cellules et les valeurs seront les cellules prédécesseurs.
        # On va mémoriser le prédécesseur de start comme étant start.
        pred = {start : start}
        
        found_stop = False
        
        # Tant qu'il reste des cellules non marquées et que l'arrivée n'a pas encore été trouvée
        while len(marked_cells) < self.height * self.width and not found_stop:
            
            # On prend la cellule au bout de la file
            current_cell = file[0]
            # et on la retire
            del file[0]

            # Si elle correspond à la cellule d'arrivée
            if current_cell == stop:
                # Terminé, on a trouvé un chemin vers la destination
                found_stop = True
            else:
                # Pour chaque voisine de la cellule
                cell_neighbors = self.get_reachable_cells(current_cell)
                
                for cell in cell_neighbors:
                    # Si elle n'est pas marquée, on la marque
                    if cell not in marked_cells:
                        marked_cells.append(cell)
                        # On la met dans la file
                        file.append(cell)
                        # Et on mémorise son prédécesseur comme étant la cellule initiale "current_cell"
                        pred[cell] = current_cell
                    
        
        # Deuxième étape: reconstruire le chemin à partir des prédécesseurs
        
        path = []
        
        # On initialise c à stop
        c = stop
        
        # Tant que c n'est pas start
        while c != start:
            # On ajoute c au chemin
            path.append(c)
            # On met le prédécesseur de c dans c
            c = pred[c]
        
        # On ajoute start dans le chemin
        path.append(start)
        
        return path

    
    def distance_geo(self, c1, c2):
        """
        Retourne la distance géodésique entre la cellule c1 et c2,
        c'est-à-dire le nombre minimal de déplacements nécessaires pour aller de c1 à c2.
        """
        return len(self.solve_dfs(c1, c2)) - 1
    
    def distance_man(self, c1, c2):
        """
        Retourne la distance de Manhattan entre la cellule c1 et c2,
        soit le nombre minimal de déplacements nécéssaires pour aller de c1 à c2
        si le labyrinthe était vide de tout mur.
        """
        # Pour déterminer cette distance, on va calculer la somme de la différence absolue
        # entre les coordonnées x des cellules et de la différence absolue entre leurs coordonnées y.
        
        return abs(c1[0] - c2[0]) + abs(c1[1] - c2[1])
    
    
    def worst_path_len(self):
        """
        Retourne la longueur du plus long chemin menant du départ (0, 0)
        à une impasse.
        """
        
        longest_path_len = 0
        
        # Pour chaque cellule, on va d'abord déterminer si elle est une impasse
        
        for i in range(self.height):
            for j in range(self.width):
                if len(self.get_reachable_cells((i, j))) == 1 and (i, j) != (0, 0):
                    # S'il s'agit d'une impasse (excepté la cellule de départ),
                    # on va calculer sa distance avec la cellule de départ et comparer avec la distance
                    # la plus longue actuellement enregistrée.
                    # Si elle est plus longue,elle devient la nouvelle plus longue distance.
                    if self.distance_geo((0, 0), (i, j)) > longest_path_len:
                        longest_path_len = self.distance_geo((0, 0), (i, j))
                    
        return longest_path_len
                    
                    
                    
    def dead_end_number(self):
        """
        Retourne le nombre de culs-de-sacs présents dans le labyrinthe.
        """
        
        # Une cellule est considérée comme un cul-de-sac s'il y a 3 murs autour d'elle.
        # Autrement dit, il faut qu'elle ne possède qu'une voisine.
        
        dead_end = 0
        
        for i in range(self.height):
            for j in range(self.width):
                if len(self.get_reachable_cells((i, j))) == 1:
                    dead_end += 1
                    
        return dead_end

### Exemple d'utilisation

In [31]:
laby = Maze(4, 4)
print(laby.info())

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente



In [32]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



### "Cassage" de murs en définissant manuellement les voisinages des cellules

In [33]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(2, 3), (0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure cohérente



### Ajout d'un mur entre la cellule (1,3) et la cellule (2,3)

In [34]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



### Test de voisinage entre ces deux cellules

In [35]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a un mur entre (1, 3) et (2, 3) car (1, 3) n'est pas dans le voisinage de (2, 3) et (2, 3) n'est pas dans le voisinage de (1, 3)


## Manipulation de labyrinthes

### Tests des méthodes:  
`remove_wall(c1, c2)`, `get_walls()`, `fill()`, `empty()`, `get_continous_ cells(c)`, `get_reachable_cells(c)`

In [36]:
laby = Maze(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [37]:
laby.empty()
print(laby)

# print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [38]:
laby.fill()
print(laby)
# print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [39]:
laby.remove_wall((0, 0), (0, 1))
print(laby)
# print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [40]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)
# print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [41]:
print(laby.get_walls())

[[(0, 0), (0, 1)], [(0, 1), (1, 1)]]


In [42]:
print(laby.get_contiguous_cells((1,1)))

[(0, 1), (2, 1), (1, 0), (1, 2)]


In [43]:
print(laby.get_reachable_cells((1,1)))

[(2, 1), (1, 0), (1, 2)]


## Génération

### Arbre binaire

In [83]:
laby = Maze.gen_btree(10, 10)
print(laby)
#print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃                   ┃           ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃   ┃       ┃       ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃   ┃               ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃                               ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃       ┃       ┃           ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃           ┃       ┃       ┃           ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃           ┃                   ┃       ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃           ┃       ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃       ┃                   ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃                                       ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



### _Sidewinder_

In [84]:
laby = Maze.gen_sidewinder(15, 15)
print(laby)
#print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃       ┃           ┃               ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃   ┃   ┃           ┃   ┃       ┃   ┃           ┃
┣   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃           ┃   ┃   ┃           ┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ┫
┃   ┃               ┃       ┃   ┃   ┃           ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃       ┃       ┃   ┃                       ┃               ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃                   ┃                   ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃       ┃   ┃           ┃   ┃                       ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃       ┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃
┣   ╋━━━

### Fusions de chemins

In [46]:
laby = Maze.gen_fusion(15, 15)
print(laby)
# print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃                           ┃           ┃       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃       ┃       ┃   ┃   ┃           ┃       ┃       ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃   ┃       ┃       ┃   ┃       ┃       ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━┫
┃       ┃                       ┃   ┃       ┃   ┃           ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━┫
┃               ┃       ┃   ┃       ┃       ┃           ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃       ┃       ┃       ┃   ┃                   ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━┫
┃       ┃   ┃       ┃   ┃       ┃   ┃   ┃   ┃   ┃   ┃       ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃   ┃               ┃   ┃   ┃   ┃           ┃   ┃
┣   ╋━━━

### Exploration exhaustive

In [47]:
laby = Maze.gen_exploration(12,15)
print(laby)
#print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃               ┃       ┃           ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━┫
┃   ┃       ┃   ┃       ┃   ┃           ┃       ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃   ┃           ┃           ┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃           ┃   ┃   ┃           ┃       ┃   ┃               ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃           ┃   ┃           ┃       ┃   ┃       ┃           ┃
┣   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃       ┃   ┃   ┃   ┃       ┃       ┃           ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━┫
┃       ┃   ┃           ┃           ┃   ┃           ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃               ┃       ┃       ┃           ┃
┣   ╋   

### Algorithme de Wilson

In [48]:
laby = Maze.gen_wilson(12, 12)
print(laby)
# print(laby.info())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃                   ┃   ┃           ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃           ┃       ┃           ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━┫
┃   ┃       ┃           ┃                   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ┫
┃                   ┃               ┃           ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃       ┃   ┃                   ┃   ┃           ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━┫
┃       ┃       ┃   ┃               ┃       ┃   ┃
┣━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃           ┃   ┃       ┃       ┃               ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃       ┃                   ┃               ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃       ┃       ┃   ┃       ┃           ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃   ┃       ┃           ┃           ┃   ┃       ┃


## Résolution

### Exemple de cas d'utilisation de `overlay`

In [49]:
laby = Maze(4,4)
laby.empty()
print(laby.overlay({
    (0, 0):'c',
    (0, 1):'o',
    (1, 1):'u',
    (2, 1):'c',
    (2, 2):'o',
    (3, 2):'u',
    (3, 3):'!'}))

┏━━━┳━━━┳━━━┳━━━┓
┃ c   o         ┃
┣   ╋   ╋   ╋   ┫
┃     u         ┃
┣   ╋   ╋   ╋   ┫
┃     c   o     ┃
┣   ╋   ╋   ╋   ┫
┃         u   ! ┃
┗━━━┻━━━┻━━━┻━━━┛



In [50]:
laby = Maze(4,4)
laby.empty()
path = {(0, 0): '@',
        (1, 0): '*',
        (1, 1): '*',
        (2, 1): '*',
        (2, 2): '*',
        (3, 2): '*',
        (3, 3): '§'}
print(laby.overlay(path))

┏━━━┳━━━┳━━━┳━━━┓
┃ @             ┃
┣   ╋   ╋   ╋   ┫
┃ *   *         ┃
┣   ╋   ╋   ╋   ┫
┃     *   *     ┃
┣   ╋   ╋   ╋   ┫
┃         *   § ┃
┗━━━┻━━━┻━━━┻━━━┛



### Résolution par parcours

In [51]:
laby = Maze.gen_fusion(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D     ┃     *   * ┃     *   *   *     ┃                   ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃ *   *   *   * ┃ *   *   * ┃     *   *         ┃       ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃       ┃       ┃           ┃   ┃ * ┃   ┃       ┃       ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃   ┃       ┃           ┃           ┃ *   *             ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃       ┃   ┃       ┃       ┃   ┃ *   *   * ┃               ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃       ┃   ┃   ┃     *   *   * ┃           ┃   ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃   ┃   ┃                   ┃           ┃ * ┃           ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━┫
┃                       ┃   ┃   ┃   ┃ *   * ┃   ┃   ┃       ┃
┣━━━╋   

In [52]:
laby = Maze.gen_exploration(15, 15)
solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *     ┃       ┃                       ┃               ┃
┣━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃ *   * ┃       ┃   ┃           ┃       ┃       ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃ * ┃           ┃   ┃   ┃   ┃       ┃       ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃ * ┃   ┃       ┃       ┃           ┃ *   * ┃   ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━┫
┃ * ┃ *   * ┃           ┃ *   *   *   * ┃ * ┃       ┃       ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃ *   * ┃ * ┃       ┃ *   *     ┃ *   * ┃ *   * ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━┫
┃   ┃ *   * ┃       ┃ *     ┃ *   * ┃ *   * ┃ * ┃   ┃       ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━┫
┃   ┃ *   *   * ┃ *   * ┃ *   * ┃         *   * ┃       ┃   ┃
┣   ╋━━━

## Évaluation

### Distance géodésique

In [53]:
laby = Maze.gen_fusion(10, 10)
c1 = (0, 0)
c2 = (9, 9)

solution = laby.solve_dfs(c1, c2)
str_solution = {c:'*' for c in solution}
str_solution[c1] = 'D'
str_solution[c2] = 'A'
print(laby.overlay(str_solution))

print(f"Distance géodésique entre {c1} et {c2} = {laby.distance_geo(c1, c2)}")
c2 = (2, 2)
print(f"Distance géodésique entre {c1} et {c2} = {laby.distance_geo(c1, c2)}")

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D ┃ *   * ┃               ┃       ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃ *   * ┃ *     ┃   ┃   ┃   ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃       ┃ * ┃   ┃ *   * ┃   ┃       ┃   ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━┫
┃   ┃   ┃ *   *   * ┃ *   *   *         ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃       ┃   ┃       ┃   ┃     *         ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃   ┃       ┃           ┃     *   *     ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━┫
┃   ┃   ┃       ┃   ┃       ┃ *   *     ┃
┣   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃   ┃             *   *     ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━┫
┃               ┃   ┃   ┃ * ┃ *   * ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃       ┃   ┃   ┃   ┃     *   * ┃ *   A ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

Distance géodésique entre (0, 0) et (9, 9) = 28
Distance géodésique entre (0, 0) et (2, 2) = 6


### Distance de Manhattan

In [54]:
laby = Maze.gen_wilson(10, 10)
c1 = (0, 0)
c2 = (9, 9)

solution = laby.solve_dfs(c1, c2)
str_solution = {c:'*' for c in solution}
str_solution[c1] = 'D'
str_solution[c2] = 'A'
print(laby.overlay(str_solution))

print(f"Distance de Manhattan entre {c1} et {c2} = {laby.distance_man(c1, c2)}")
c2 = (2, 2)
print(f"Distance de Manhattan entre {c1} et {c2} = {laby.distance_man(c1, c2)}")

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   * ┃   ┃       ┃               ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃     *     ┃   ┃           ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━┫
┃       ┃ *         ┃   ┃               ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━┫
┃   ┃ *   * ┃           ┃   ┃           ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃   ┃ *   * ┃       ┃               ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃     * ┃               ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ┫
┃         * ┃   ┃       ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃ * ┃           ┃   ┃   ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━┫
┃       ┃ *     ┃     *   *   * ┃       ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃         *   *   *   *     ┃ *   *   A ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

Distance de Manhattan entre (0, 0) et (9, 9) = 18
Distance de Manhattan entre (0, 0) et (2, 2) = 4


### Plus long chemin entre le départ et une impasse

In [55]:
laby = Maze.gen_exploration(5, 5)
print(laby)
print(f"Longueur du plus long chemin entre le départ et une impasse : {laby.worst_path_len()}")

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃   ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃       ┃   ┃       ┃
┣━━━╋━━━╋   ╋━━━╋━━━┫
┃       ┃           ┃
┣   ╋━━━╋   ╋━━━╋   ┫
┃       ┃       ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛

Longueur du plus long chemin entre le départ et une impasse : 15


### Nombre de culs-de-sacs

In [56]:
laby = Maze.gen_wilson(5, 5)
print(laby)
print(f"Nombre de culs-de-sacs: {laby.dead_end_number()}")

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣━━━╋━━━╋━━━╋   ╋   ┫
┃       ┃       ┃   ┃
┣   ╋━━━╋   ╋   ╋━━━┫
┃           ┃   ┃   ┃
┣━━━╋   ╋   ╋   ╋   ┫
┃       ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ┫
┃       ┃           ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛

Nombre de culs-de-sacs: 7
